In [ ]:
"""

Author : Elsie Rezinold Yedida


"""
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

import matplotlib.pyplot as plt

from tensorflow.keras import layers
from tensorflow.keras import models

(train,test),info = tfds.load('mnist',split=
		['train','test'],
		with_info=True,
		as_supervised=True
		)


def format_image(image,label):
	image = tf.image.resize(image,(16,16))/255.0
	return image,label

BATCH_SIZE = 32
num_examples = info.splits['train'].num_examples

train_ex = train.cache().shuffle(num_examples//4).map(format_image).batch(BATCH_SIZE).prefetch(1)
test_ex = test.cache().map(format_image).batch(BATCH_SIZE).prefetch(1)

layer0 = layers.Flatten(input_shape=(16,16))
layer1 = layers.Dense(20,activation='relu')
#layer1_1 = layers.Dense(20,activation='relu')
#layer2 = layers.Dense(10,activation='softmax')
layer2 = layers.Dense(10,activation='sigmoid')

model = models.Sequential([layer0,layer1,layer2])
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['acc'])
history =model.fit(train_ex,epochs=100)
model.evaluate(test_ex)
acc = history.history['acc']

epochs_range = range(100)

loss = history.history['loss']
plt.figure(figsize=(8,8))
plt.subplot(1,2,1)
plt.plot(epochs_range,acc,label='Training_accuracy')
#plt.plot(epochs_range,val_acc,label='validation_accuracy')
plt.legend(loc='lower right')
plt.title('Training & Validation Accuracy')

plt.subplot(1,2,2)
plt.plot(epochs_range,loss,label='loss')
#plt.plot(epochs_range,val_loss,label='validation_loss')
plt.legend(loc='upper right')
plt.title('Training & Validation loss')
plt.savefig('./metrics.png')
plt.show()




In [ ]:
test_im,test_lb = next(iter(test_ex.take(1)))
test_im =test_im.numpy()
test_lb = test_lb.numpy()

im0 = test_im[:32]
lb0 = test_lb[:32]

im0 = tf.squeeze(im0).numpy()
args = model.predict(im0)
args = tf.squeeze(args).numpy()

prediction_ids = np.argmax(args,axis=-1)
print(prediction_ids)
print(lb0)








In [ ]:
import nn_functions as nn
print(im0.shape)
im0=im0.reshape(32,256)
im0 = np.transpose(im0)
print(im0.shape)

w0 = np.zeros((20,256))
w0 = np.transpose(model.layers[1].get_weights()[0])
w1 = np.zeros((10,20))
w1 = np.transpose(model.layers[2].get_weights()[0])
b0 = np.zeros((20,1))
print (b0.shape)
b0 = model.layers[1].get_weights()[1]
print (b0.shape)
b0=b0.reshape(20,1)
print (b0.shape)
b1 = np.zeros((10,1))
b1 = model.layers[2].get_weights()[1]
print (b1.shape)
b1=b1.reshape(10,1)
print (b1.shape)
y1 = np.zeros((20,32))
y2 = np.zeros((10,32))
y2_ws = np.zeros((10,32))

y1 = nn.node(im0,w0,b0,"relu")
y2 = nn.node(y1,w1,b1,"sigmoid")
y2_ws = nn.node(y1,w1,b1,"")

print(y2_ws.shape)

In [ ]:
print(args[0])
y2 = np.transpose(y2)
y2_ws = np.transpose(y2_ws)
print(y2[0])
print(y2_ws[0])

In [ ]:
import bitstring, random
def ieee754(flt):
    b = bitstring.BitArray(float=flt, length=32)
    return b


In [ ]:
for i in range(10):
    num = ieee754(y2_ws[0][i])
    print(num.hex)

    

In [ ]:
fl = """ file location"""
with open(fl,'w') as f:
    f.write("initial begin\n")
    for j in range(20):
        for i in range(256):
            num = ieee754(w0[j][i])
            f.write("\tw1_"+str(j+1)+"["+str(32*(i+1) -1)+":"+str(32*i)+"] = 32'h" +num.hex+";\n")
    f.write("end\n")



In [ ]:
fl = """ file location"""
with open(fl,'w') as f:
    f.write("initial begin\n")
    for j in range(10):
        for i in range(20):
            num = ieee754(w1[j][i])
            f.write("\tw2_"+str(j+1)+"["+str(32*(i+1) -1)+":"+str(32*i)+"] = 32'h" +num.hex+";\n")
    f.write("end\n")



In [ ]:
fl = """ file location"""
with open(fl,'w') as f:
    f.write("initial begin\n")
    for i in range(20):
        num = ieee754(b0[i])
        f.write("\tb1["+str(32*(i+1) -1)+":"+str(32*i)+"] = 32'h" +num.hex+";\n")
    f.write("end\n")



In [ ]:
fl = """ file location"""
with open(fl,'w') as f:
    f.write("initial begin\n")
    for i in range(10):
        num = ieee754(b1[i])
        f.write("\tb2["+str(32*(i+1) -1)+":"+str(32*i)+"] = 32'h" +num.hex+";\n")
    f.write("end\n")
        

In [ ]:

fl = """ file location"""
with open(fl,'w') as f:
    f.write("initial begin\n")
    for j in range(32):
        k = 0
        for i in range(256):
            num = ieee754(im0[i][j])
            f.write("\tin"+"["+str(j)+"]"+"["+str(32*(k+1) -1)+":"+str(32*k)+"] = 32'h" +num.hex+";\n")
            k +=1
    f.write("end\n")

    


In [ ]:

fl = """ file location"""
with open(fl,'w') as f:
    for j in range(32):
        for i in range(20):
            num = ieee754(y1[i][j])
            f.write(num.hex+"\n")




In [ ]:

fl = """ file location"""
with open(fl,'w') as f:
    for i in range(32):
        for j in range(10):
            num = ieee754(y2_ws[i][j])
            f.write(num.hex+"\n")

In [ ]:
test_batch = test.cache().map(format_image).batch(10000).prefetch(1)

test_im,test_lb = next(iter(test_batch.take(1)))
test_im =test_im.numpy()
test_lb = test_lb.numpy()

im0 = test_im[:10000]
lb0 = test_lb[:10000]

im0 = tf.squeeze(im0).numpy()
args = model.predict(im0)
args = tf.squeeze(args).numpy()

prediction_ids = np.argmax(args,axis=-1)
print(prediction_ids)
print(lb0)


In [ ]:
import nn_functions as nn
print(im0.shape)
im0=im0.reshape(10000,256)
im0 = np.transpose(im0)
print(im0.shape)
"""
w0 = np.zeros((20,256))
w0 = np.transpose(model.layers[1].get_weights()[0])
w1 = np.zeros((10,20))
w1 = np.transpose(model.layers[2].get_weights()[0])
b0 = np.zeros((20,1))
print (b0.shape)
b0 = model.layers[1].get_weights()[1]
print (b0.shape)
b0=b0.reshape(20,1)
print (b0.shape)
b1 = np.zeros((10,1))
b1 = model.layers[2].get_weights()[1]
print (b1.shape)
b1=b1.reshape(10,1)
print (b1.shape)
y1 = np.zeros((20,32))
y2 = np.zeros((10,32))
y2_ws = np.zeros((10,32))
"""
y1 = nn.node(im0,w0,b0,"relu")
y2 = nn.node(y1,w1,b1,"sigmoid")
y2_ws = nn.node(y1,w1,b1,"")

print(y2_ws.shape)

In [ ]:
fl = """ file location"""
with open(fl,'w') as f:
    f.write("initial begin\n")
    for j in range(10000):
        k = 0
        for i in range(256):
            num = ieee754(im0[i][j])
            f.write("\tin"+"["+str(j)+"]"+"["+str(32*(k+1) -1)+":"+str(32*k)+"] = 32'h" +num.hex+";\n")
            k +=1
    f.write("end\n")


In [ ]:
fl = """ file location"""
with open(fl,'w') as f:
    for j in range(10000):
        for i in range(20):
            num = ieee754(y1[i][j])
            f.write(num.hex+"\n")


In [ ]:
y2_ws = np.transpose(y2_ws)
fl = """ file location"""
with open(fl,'w') as f:
    for i in range(10000):
        for j in range(10):
            num = ieee754(y2_ws[i][j])
            f.write(num.hex+"\n")